<a href="https://colab.research.google.com/github/FurrukhJamal/ColabML/blob/main/Mnist_using_sequectial_and_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
assert sys.version_info >= (3, 7)

sys.version_info(major=3, minor=11, micro=11, releaselevel='final', serial=0)

In [ ]:
from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.0")

installing Keras Tuner library

In [ ]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt
import tensorflow as tf
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train.shape
Y_train.shape
X_test.shape
Y_test.shape


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


(10000,)

In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
X_test.shape

(10000, 28, 28)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(48000, 28, 28)

In [ ]:
X_val.shape

(12000, 28, 28)

In [22]:
def model_build(hp):
  n_hidden = hp.Int("n_hidden", min_value = 0, max_value = 8, default = 2)
  n_neurons = hp.Int("n_neurons", min_value = 16, max_value = 300)
  learning_rate = hp.Float("learning_rate", min_value = 1e-4, max_value = 1e-1, sampling = "log")
  optimizer = hp.Choice("optimizer", values = ["sgd", "adam"])
  if optimizer == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
  else:
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape= [28, 28]))
  for _ in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_neurons, activation = "relu"))
  model.add(tf.keras.layers.Dense(10, activation = "softmax"))

  model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

  return model

In [23]:
random_search_tuner = kt.RandomSearch(model_build, objective = "val_accuracy", max_trials = 5, overwrite = True, directory = "my first test", project_name = "mnist", seed = 42)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience = 4, restore_best_weights = True)

random_search_tuner.search(X_train, Y_train, epochs = 100, validation_data = (X_val, Y_val))

Trial 5 Complete [00h 12m 44s]
val_accuracy: 0.8735833168029785

Best val_accuracy So Far: 0.8861666917800903
Total elapsed time: 01h 03m 32s


In [24]:
top3_models = random_search_tuner.get_best_models(num_models = 3)
best_model = top3_models[0]


In [25]:
best_hyperparameters = random_search_tuner.get_best_hyperparameters(num_trials = 1)[0]
best_hyperparameters.values

{'n_hidden': 7,
 'n_neurons': 116,
 'learning_rate': 0.004410354359965501,
 'optimizer': 'sgd'}

In [26]:
test_loss, test_accuracy = best_model.evaluate(X_test, Y_test)
print(f"test_loss : {test_loss} test_accuracy : {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8806 - loss: 0.4330
test_loss : 0.4352591335773468 test_accuracy : 0.8802000284194946
